In [1]:
from cimaq_utils import *
from cimaq_utils import loadimages
from blind_rename import *
from inspect_misc_text import *
from zipctl import uzipfiles


In [ ]:
new_indiv_sheets_infos = sortmap(loadfiles(new_events_dir), indiv_patterns)
ind_pars_infos = df(get_infos(fpath) for fpath in new_indiv_sheets_infos.fpaths)

In [18]:
new_indiv_sheets_infos['pscid'] = [[int(itm) for itm in fname.split('_')
                                    if len(itm) == 7 and itm.isnumeric()]
                                   for fname in new_indiv_sheets_infos.fname]
new_indiv_sheets_infos['pscid'] = [float(itm[0]) if itm else None for itm
                                   in new_indiv_sheets_infos.pscid]
new_confounds = loadfiles(new_conf_dir)

In [20]:
onset_event = new_indiv_sheets_infos.loc[[row[0] for row in new_indiv_sheets_infos.iterrows()
                                       if row[1]['onset_event']]].reset_index(drop=True)
output_retrieval = new_indiv_sheets_infos.loc[[row[0] for row in new_indiv_sheets_infos.iterrows()
                                       if row[1]['output_retrieval']]].reset_index(drop=True)
output_responses = new_indiv_sheets_infos.loc[[row[0] for row in new_indiv_sheets_infos.iterrows()
                                       if row[1]['output_responses']]].reset_index(drop=True)

all_new_mean_sheets = tuple(pd.read_csv(fpath).drop(columns='Unnamed: 0')
                            for fpath in new_mean_sheets.fpaths)
shortest_sheet = get_shortest_ind(all_new_mean_sheets)
shortest_sheet['dccid'] = [int(num_only(str(dccid))) for dccid in shortest_sheet.dccid]
shortest_sheet = shortest_sheet.set_index('dccid')
all_dccids = shortest_sheet.index

all_new_mean_sheets = [sheet.set_index('dccid').loc[all_dccids].sort_index(
                          ).reset_index(drop=False).convert_dtypes(float)
                              for sheet in all_new_mean_sheets]

renamer = pd.concat([all_new_mean_sheets[1]['pscid'], all_new_mean_sheets[1]['dccid']], axis=1)
renamer['pscid'] = [float(num_only(str(subid))) for subid in renamer.pscid]
renamer['dccid'] = [float(num_only(str(subid))) for subid in renamer.dccid]
renamer['dccid'] = renamer['dccid'].convert_dtypes(float)
all_new_mean_sheets = tuple(pd.merge(sheet, renamer, on='dccid', how='outer')
                            for sheet in all_new_mean_sheets)

mean_sheets_patterns2 = df(mean_sheets_patterns,
                           columns=['ids', 'patterns'])
new_confounds['dccid'] = [int(num_only(str(fname.split('sub')[1][:6])))
                          for fname in new_confounds.fname]
new_confounds = new_confounds.set_index('dccid').loc[all_dccids].sort_index().reset_index()
anat_scans_infos['dccid'] = [int(num_only(str(fname.split('sub')[1][:6])))
                             for fname in anat_scans_infos.fname]
anat_scans_infos = anat_scans_infos.set_index('dccid').loc[all_dccids].sort_index().reset_index()
fmri_scans_infos['dccid'] = [int(num_only(str(fname.split('sub')[1][:6])))
                             for fname in fmri_scans_infos.fname]
fmri_scans_infos = fmri_scans_infos.set_index('dccid').loc[all_dccids].sort_index().reset_index()

new_confounds = pd.merge(new_confounds, renamer, on='dccid', how='outer')

anat_scans_infos = pd.merge(anat_scans_infos, renamer, on='dccid', how='outer')
fmri_scans_infos = pd.merge(fmri_scans_infos, renamer, on='dccid', how='outer')

onset_event['pscid'] = [int(fname.split('_')[4]) for fname in onset_event.fname]
output_retrieval['pscid'] = [int(fname.split('_')[3]) for fname in output_retrieval.fname]
output_responses['pscid'] = [int(fname.split('_')[4]) for fname in output_responses.fname]

onset_event = onset_event.set_index('pscid').sort_index()
onset_event.index.dtypes = np.int64
output_retrieval.index.dtypes, output_responses.index.dtypes = np.int64, np.int64

renamer = renamer.astype(int)

output_retrieval = output_retrieval.set_index('pscid').sort_index()
output_responses = output_responses.set_index('pscid').sort_index()

onset_event = pd.merge(onset_event, renamer, on='pscid', how='outer')
output_retrieval = pd.merge(output_retrieval, renamer, on='pscid', how='outer')
output_responses = pd.merge(output_responses, renamer, on='pscid', how='outer')

onset_event = onset_event.dropna()
output_retrieval = output_retrieval.dropna()
output_responses = output_responses.dropna()

allinfos = [new_confounds, onset_event, output_retrieval, output_responses]
allinfos = [sheet.set_index('dccid') for sheet in allinfos]

allinds = pd.Series(flatten(list(sheet.dropna(axis=0, how='any').index) for sheet in allinfos)).unique()
shrt = [int(itm) for itm in get_shortest_ind(allinfos).dropna(axis=0, how='any').index
        if itm in allinds]
allinfos = [sheet.loc[[ind for ind in shrt if ind in
                       list(sheet.index)]].dropna(axis=1,how='any')
            for sheet in allinfos]
allfiles = pd.concat([sheet for sheet in allinfos]).fillna(False)
allfiles['confounds'] = ['confounds' in fname for fname in allfiles.fname]
allfiles['pscid'] = [int(ids) for ids in allfiles['pscid']]



In [49]:
allfiles.to_csv(xpu('~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/sorted_files.csv'))

In [78]:
big_infos = df(get_infos(str(fpath)) for fpath in allfiles.fpaths)

In [58]:
allsheets = tuple((fpath, pd.read_csv(fpath).drop(columns='Unnamed: 0'))
                  for fpath in allfiles.fpaths)

In [71]:
test = pd.concat([allfiles.loc[[row[0] for row in allfiles.iterrows()
                               if row[1]['onset_event']]].sample(),
        allfiles.loc[[row[0] for row in allfiles.iterrows()
                               if row[1]['output_responses']]].sample(),
        allfiles.loc[[row[0] for row in allfiles.iterrows()
                               if row[1]['output_retrieval']]].sample(),
        allfiles.loc[[row[0] for row in allfiles.iterrows()
                               if row[1]['confounds']]].sample()])

In [76]:
test_sheets = tuple((fpath, pd.read_csv(fpath).drop(columns='Unnamed: 0')) for fpath in test.fpaths)

In [77]:
display([sheet[1] for sheet in test_sheets])

[       0    1       2      3         4        5    6         7        8     9
 0      1  enc   enc00    NaN  encoding    0.069  3.0  fixation    3.069   4.5
 1      2  enc  enc000    NaN  encoding    7.568  3.0  fixation   10.568   0.5
 2      3  ctl    ctl0    NaN   control   11.068  3.0  fixation   14.068   0.5
 3      4  enc   enc01  old55  encoding   14.568  3.0  fixation   17.567   6.5
 4      5  enc   enc02  old05  encoding   24.067  3.0  fixation   27.066   2.0
 ..   ...  ...     ...    ...       ...      ...  ...       ...      ...   ...
 115  116  enc   enc74  old48  encoding  710.003  3.0  fixation  713.002   4.5
 116  117  enc   enc75  old51  encoding  717.502  3.0  fixation  720.502   5.5
 117  118  enc   enc76  old20  encoding  726.001  3.0  fixation  729.001   1.0
 118  119  enc   enc77  old29  encoding  730.001  3.0  fixation  733.001   0.5
 119  120  enc   enc78  old17  encoding    733.5  3.0  fixation    736.5  13.5
 
 [120 rows x 10 columns],
        0    1       2  